In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers

# a. Load pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:   # b. Freeze base layers
    layer.trainable = False

# c. Prepare data
data_dir = r'C:/Users/UMESH/OneDrive/Desktop/LP_4'
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train = datagen.flow_from_directory(data_dir, target_size=(224, 224), subset='training')
val = datagen.flow_from_directory(data_dir, target_size=(224, 224), subset='validation')

# d. Add classifier and train
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train, epochs=5, validation_data=val)

# e. Fine-tune few layers
for layer in base_model.layers[-4:]:
    layer.trainable = True
model.compile(optimizer=optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train, epochs=3, validation_data=val)

model.save('accordion_model.h5')

Found 7357 images belonging to 102 classes.
Found 1788 images belonging to 102 classes.
9406464/9406464 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                          